# Text Data Processing with PyTorch

[Original tutorial](https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84)

In [1]:
import os
import re
import spacy
import torch

import numpy as np
import pandas as pd

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import (
    Dataset
    , DataLoader
    , random_split
)

sys.path.insert(0, ".")
from load import *

In [ ]:
# This is a small subset of the full toxic comment dataset
df_sample = pd.read_csv("./sample.csv", index_col="id")
df_sample.head(5)

In [2]:
x = ToxicComtDataset("sample.csv")

In [3]:
v = x.get_vocab()

In [4]:
v.text_to_num("How do you do?")

[210, 24, 11, 24, 26]

In [5]:
type(v)

load.ToxicComtVocab